In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/RecFldGrn


# PreCode

In [2]:
import os
from recfldgrn.datapoint import load_df_data_with_RecName, DataPoint
IDName = 'PID'; RANGE_SIZE = 10000

######################
rec_folder = 'data/ProcData/RecFolder'
####################

RecName_List = os.listdir(rec_folder)
print(RecName_List)
IDValue = 'P5'   # PhyID value
# print(data_folder)
Pat = DataPoint(IDName, IDValue, rec_folder, RANGE_SIZE)
# print(Pat)
dfx = Pat.get_df_rec('P')
df = load_df_data_with_RecName(rec_folder, 'P')
# dfx

['PNSect', 'Diag', 'Smoking', 'EC', 'PN', 'A1C', 'PNSectSent', 'P']


# P@PatEcInfo-InfoGrn

## df_whole/row

### Settings

In [3]:
###################
RecName_Chain = ['P'] 
RecTableName2FldColumns_Dict = {
    # The first should be in the level of RecName_Chain[-1]
    'P': ['PID', 'age', 'basicInfo'],
    'EC': ['DT_min', 'DT_max'],
    # TODO: to add the df_whole filter or dp filter, you may want to add the EC DT information. 
}
###################

########################
## Prediction Index Date
RecName = RecName_Chain[-1] # row level
SynFld = 'PECInfo' # This name must be special
GrnName = 'InfoGrn'
########################

###################
rec_folder = 'data/ProcData/RecFolder/'
###################

###################
reshape_fn_kwargs = {}
###################


### Load df_whole/row

In [4]:
from recfldgrn.loadtools import get_df_whole_from_settings, get_df_individual_from_settings

recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
full_recfldgrn = f'{"-".join(RecName_Chain)}@{SynFld}-{GrnName}'

df_whole = get_df_whole_from_settings(RecName_Chain, RecTableName2FldColumns_Dict, rec_folder)
# df_whole = get_df_individual_from_settings(RecName_Chain,  RecTableName2FldColumns_Dict, Pat,)
row = df_whole.iloc[0]

### Filter df_whole/row (optional)

In [5]:
import pandas as pd

for Rec in RecTableName2FldColumns_Dict:
    print(f'\n{Rec} =======')
    s = df_whole[Rec].apply(lambda x: len(x) if type(x) == type(pd.DataFrame()) else 'Missing')
    print(s.value_counts().sort_index())


P =======
1    8
Name: P, dtype: int64

EC =======
1     1
7     1
22    1
23    1
25    1
26    1
34    1
87    1
Name: EC, dtype: int64


## grain_tkn_fn

### Function Development

In [6]:
import inspect
from recfldgrn.graintools import func_convert_Num_factory

def grain_tkn_fn(row):
    import itertools
    
    buffer_dict = {}
    
    # Patient Part: 
    df_recinput = row['P'][['age', 'basicInfo']] # this special because xxx.
    
    # P@Age
    fld = 'age'
    start, end, Min, Max, scale = [18, 80, 1, 120, 5]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = df_recinput[fld].iloc[0]
    results = func_convert_Num_wgt(value)
    li_value_tkn = [f'{fld}_{i}' for i in results[0]]
    li_value_wgt = results[1]
    # print(len(li_value_tkn), len(li_value_wgt))
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # P@basicInfo
    fld = 'basicInfo'
    basicInfo_series = df_recinput[fld]# .mean()
    value = basicInfo_series.iloc[0]
    li_value_tkn = value.split('&')
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # EC Part
    df_recinput = row['EC'][['DT_min', 'DT_max']]
    
    # P@ECNum
    fld = 'ecNum'
    value = len(df_recinput)
    li_value_tkn = [f'{fld}_{value}' ]
    li_value_wgt = [1] * len(li_value_tkn)
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # P@ecDays
    fld = 'ecDays'
    start, end, Min, Max, scale = [0, 100, 0, 500, 7]
    func_convert_Num_wgt = func_convert_Num_factory(start, end, Min, Max, scale)
    value = (df_recinput['DT_max'] - df_recinput['DT_min']).mean().total_seconds() / 60 / 60 / 24
    results = func_convert_Num_wgt(value)
    li_value_tkn = [f'{fld}{i}' for i in results[0]]
    li_value_wgt = results[1]
    li_value_fld = [fld] * len(li_value_tkn)
    buffer_dict[fld] = {'key': li_value_tkn, 'wgt': li_value_wgt, 'tpc':li_value_fld}
    
    # summary
    output = {}
    for i in ['key', 'wgt', 'tpc']:
        # output[i] = sum(, [])
        output[i.split('_')[-1]] = list(itertools.chain(*[d[i] for fld, d in buffer_dict.items()]))
        
    # key: tkn
    # value: wgt
    # fld: hypertype?
    return output

func_convert_Num_factory.fn_string = inspect.getsource(func_convert_Num_factory)
grain_tkn_fn.fn_string = inspect.getsource(grain_tkn_fn)
# print(grain_str_fn_string)

### Test

In [7]:
output = grain_tkn_fn(row)
for name, grn_list in output.items():
    print('\n')
    print(name)
    print(grn_list)
    print(len(grn_list))



key
['age_miss', 'age_more', 'age_less', 'age_bottom', 'age_top', 'age_base', 'age_B18+', 'age_B23+', 'age_B28+', 'age_B33+', 'age_B38+', 'age_B43+', 'age_B48+', 'age_B53+', 'age_B58+', 'age_B63+', 'age_B68+', 'age_B73+', 'age_B78+', 'Male', 'B', 'ecNum_34', 'ecDaysmiss', 'ecDaysmore', 'ecDaysless', 'ecDaysbottom', 'ecDaystop', 'ecDaysbase', 'ecDaysB0+', 'ecDaysB7+', 'ecDaysB14+', 'ecDaysB21+', 'ecDaysB28+', 'ecDaysB35+', 'ecDaysB42+', 'ecDaysB49+', 'ecDaysB56+', 'ecDaysB63+', 'ecDaysB70+', 'ecDaysB77+', 'ecDaysB84+', 'ecDaysB91+', 'ecDaysB98+']
43


wgt
[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1.0, 1.0, 0.0126, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
43


tpc
['age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'age', 'basicInfo', 'basicInfo', 'ecNum', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 'ecDays', 

In [8]:
recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
df_whole[recfldgrn] = df_whole.apply(lambda row: grain_tkn_fn(row), axis = 1)
df_fld = df_whole[[R+'ID' for R in RecName_Chain] + [recfldgrn]]
df_fld

,PID,P@PECInfo-InfoGrn
0,P0,"{'key': ['age_miss', 'age_more', 'age_less', '..."
1,P1,"{'key': ['age_miss', 'age_more', 'age_less', '..."
2,P2,"{'key': ['age_miss', 'age_more', 'age_less', '..."
3,P3,"{'key': ['age_miss', 'age_more', 'age_less', '..."
4,P4,"{'key': ['age_miss', 'age_more', 'age_less', '..."
5,P5,"{'key': ['age_miss', 'age_more', 'age_less', '..."
6,P6,"{'key': ['age_miss', 'age_more', 'age_less', '..."
7,P7,"{'key': ['age_miss', 'age_more', 'age_less', '..."


## Save PyFile

### Prepare Variables

In [9]:
string_variables = [RecName, SynFld, GrnName, rec_folder]
iterative_variables = [RecName_Chain, RecTableName2FldColumns_Dict, reshape_fn_kwargs]
fn_variables = [grain_tkn_fn, func_convert_Num_factory]
prefix = ['import pandas as pd', 'import numpy as np', 'import itertools']

### Save to PyFile

In [10]:
from recfldgrn.loadtools import retrieve_name, convert_variables_to_pystirng

python_strings  = convert_variables_to_pystirng(string_variables, iterative_variables, fn_variables, prefix)
Meta_folder = rec_folder.replace('RecFolder', 'FldGrnMeta')
meta_file = os.path.join(Meta_folder, SynFld + '.py')
print(meta_file)
with open(meta_file, 'w') as f: 
    f.write(python_strings)

data/ProcData/FldGrnMeta/PECInfo.py


# Application

In [1]:
# restart
%cd ..

/Users/floydluo/Library/CloudStorage/OneDrive-JohnsHopkins/000Projects/0000-Infrastructure/0000-RecFld/RecFldGrn


## Settings

In [2]:
##########
rec_folder = 'data/ProcData/RecFolder/'
SynFld = 'PECInfo' # ----> get its MetaDict
##########


# filtering hyperparameters
######### get df_whole: obsind, predind, and futind
PpredTable = 'xxx'
PpredColum = 'xxx'
Range_List = ['Bf2y']
#########

## Load FldGrn PyFile

In [3]:
import os
import pandas as pd
from recfldgrn.loadtools import load_module_variables

Meta_folder = rec_folder.replace('RecFolder', 'FldGrnMeta')
meta_file = os.path.join(Meta_folder, SynFld + '.py')
module = load_module_variables(meta_file)
MetaDict = module.MetaDict

RecName = MetaDict['RecName']#  'P'
RecName_Chain = MetaDict['RecName_Chain'] # ['P']
# RecName_Chain_to_RecName = MetaDict['RecName_Chain_to_RecName'] # {} # Can be updated further. 
RecTableName2FldColumns_Dict = MetaDict['RecTableName2FldColumns_Dict']
# SynFld = MetaDict['SynFld'] # 'PECINfo'
GrnName = MetaDict['GrnName'] # 'InfoGrn'
# sfx_info = MetaDict['sfx_info']
# rec_folder = MetaDict['rec_folder'] # 'data/ProcData/RecFolder/'
reshape_fn_kwargs = MetaDict['reshape_fn_kwargs']
grain_tkn_fn = MetaDict['grain_tkn_fn']

recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
recfldgrn

'P@PECInfo-InfoGrn'

##  Load & Filter df_whole/row

In [4]:
RecTableName2FldColumns_Dict = RecTableName2FldColumns_Dict # TODO, to update.
# RecTableName2FldColumns_Dict[PpredTable] = [PpredColum]

In [5]:
from recfldgrn.loadtools import get_df_whole_from_settings, get_df_individual_from_settings
df_whole = get_df_whole_from_settings(RecName_Chain, RecTableName2FldColumns_Dict,  rec_folder)

In [6]:
## filtering

## Generate RecFldGrn

In [7]:
recfldgrn = f'{RecName}@{SynFld}-{GrnName}'
df_whole[recfldgrn] = df_whole.apply(lambda row: grain_tkn_fn(row), axis = 1)
df_fld = df_whole[[R+'ID' for R in RecName_Chain] + [recfldgrn]].reset_index(drop = True)
df_fld

,PID,P@PECInfo-InfoGrn
0,P0,"{'key': ['age_miss', 'age_more', 'age_less', '..."
1,P1,"{'key': ['age_miss', 'age_more', 'age_less', '..."
2,P2,"{'key': ['age_miss', 'age_more', 'age_less', '..."
3,P3,"{'key': ['age_miss', 'age_more', 'age_less', '..."
4,P4,"{'key': ['age_miss', 'age_more', 'age_less', '..."
5,P5,"{'key': ['age_miss', 'age_more', 'age_less', '..."
6,P6,"{'key': ['age_miss', 'age_more', 'age_less', '..."
7,P7,"{'key': ['age_miss', 'age_more', 'age_less', '..."


## Save FldGrn Data (Optional)

# Dataset Construction

## Track 1: df_table (ml)

In [8]:
# Your input here: you can have multiple df_fld, actually. 
print(df_fld.shape)
df_fld.head()

(8, 2)


,PID,P@PECInfo-InfoGrn
0,P0,"{'key': ['age_miss', 'age_more', 'age_less', '..."
1,P1,"{'key': ['age_miss', 'age_more', 'age_less', '..."
2,P2,"{'key': ['age_miss', 'age_more', 'age_less', '..."
3,P3,"{'key': ['age_miss', 'age_more', 'age_less', '..."
4,P4,"{'key': ['age_miss', 'age_more', 'age_less', '..."


### prepare df_ml

In [9]:
# TODO: this need further operations
df_ml = df_fld.copy()

In [10]:
df_value = pd.DataFrame(df_ml[recfldgrn].apply(lambda x: dict(zip(x['key'], x['wgt']))).to_list())
df_table = pd.concat([df_ml[[R+'ID' for R in RecName_Chain]], df_value], axis = 1)
df_table

,PID,age_miss,age_more,age_less,age_bottom,age_top,age_base,age_B18+,age_B23+,age_B28+,...,Female,ecNum_26,ecNum_87,A,ecNum_22,D,ecNum_23,ecNum_7,C,ecNum_1
0,P0,0,0,0,0,0,1,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P1,0,0,0,0,0,1,0.6,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P2,0,0,0,0,0,1,1.0,0.0,0.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P3,0,0,0,0,0,1,1.0,1.0,1.0,...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P4,0,0,0,0,0,1,1.0,0.0,0.0,...,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN
5,P5,0,0,0,0,0,1,1.0,1.0,1.0,...,1.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
6,P6,0,0,0,0,0,1,1.0,1.0,1.0,...,1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN
7,P7,0,0,0,0,0,1,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


## Track 2: df_tensor

### prepare df_fld

In [11]:
# Your input here: you can have multiple df_fld, actually. 
print(df_fld.shape)
df_fld.head()

(8, 2)


,PID,P@PECInfo-InfoGrn
0,P0,"{'key': ['age_miss', 'age_more', 'age_less', '..."
1,P1,"{'key': ['age_miss', 'age_more', 'age_less', '..."
2,P2,"{'key': ['age_miss', 'age_more', 'age_less', '..."
3,P3,"{'key': ['age_miss', 'age_more', 'age_less', '..."
4,P4,"{'key': ['age_miss', 'age_more', 'age_less', '..."


### get df_tensor

In [12]:
# TODO: this need further operations
df_tensor = df_fld.copy()

In [13]:
# we can save it for the future. 

for i in ['key', 'tpc', 'wgt']: 
    df_tensor[f'{recfldgrn}_{i}'] = df_tensor[recfldgrn].apply(lambda x: x[i])
    
df_tensor = df_tensor.drop(columns = [recfldgrn])
df_tensor

,PID,P@PECInfo-InfoGrn_key,P@PECInfo-InfoGrn_tpc,P@PECInfo-InfoGrn_wgt
0,P0,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
1,P1,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0...."
2,P2,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
3,P3,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
4,P4,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0...."
5,P5,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."
6,P6,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0...."
7,P7,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1...."


### get idx and EmbedDict

In [14]:
import pandas as pd 
from recfldgrn.graintools import generate_grain_vocab_info

def generate_EmbedDict_from_processed_df_whole(df_whole, recfldgrn, sfx_list = ['key', 'tpc']):
    EmbedDict = {}
    
    for sfx in sfx_list:
        embed_type = 'CateEmbed'
        recfldgrn_sfx = f'{recfldgrn}_{sfx}'
        # print(recfldgrn_sfx)
        s = df_whole[recfldgrn_sfx] 
        Vocab = generate_grain_vocab_info(s) 
        v2idx = Vocab['v2idx']
        vocab_size = len(v2idx)
        # print(v2idx)
        d = {'embed_type':embed_type, 
             'recfldgrn_sfx':recfldgrn_sfx,
             'vocab_size': vocab_size,
             'Vocab': Vocab}
        
        df_whole[f'{recfldgrn}_{sfx}idx'] =  df_whole[f'{recfldgrn}_{sfx}'].apply(lambda x: [v2idx[i] for i in x])
        EmbedDict[sfx] = d
        
    return df_whole, EmbedDict


In [15]:
df_tensor, EmbedDict = generate_EmbedDict_from_processed_df_whole(df_tensor, recfldgrn)
df_tensor.head()

,PID,P@PECInfo-InfoGrn_key,P@PECInfo-InfoGrn_tpc,P@PECInfo-InfoGrn_wgt,P@PECInfo-InfoGrn_keyidx,P@PECInfo-InfoGrn_tpcidx
0,P0,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,P1,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,P2,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,P3,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,P4,"[age_miss, age_more, age_less, age_bottom, age...","[age, age, age, age, age, age, age, age, age, ...","[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [16]:
print(EmbedDict)

{'key': {'embed_type': 'CateEmbed', 'recfldgrn_sfx': 'P@PECInfo-InfoGrn_key', 'vocab_size': 55, 'Vocab': {'idx2v': {0: '_padding', 1: 'age_miss', 2: 'ecDaysmiss', 3: 'ecDaysless', 4: 'ecDaysbottom', 5: 'ecDaystop', 6: 'age_more', 7: 'ecDaysB0+', 8: 'ecDaysB7+', 9: 'ecDaysB14+', 10: 'ecDaysB21+', 11: 'ecDaysB28+', 12: 'ecDaysB35+', 13: 'ecDaysB42+', 14: 'ecDaysB49+', 15: 'ecDaysB56+', 16: 'ecDaysB63+', 17: 'ecDaysB70+', 18: 'ecDaysB77+', 19: 'ecDaysB84+', 20: 'ecDaysB91+', 21: 'ecDaysB98+', 22: 'ecDaysmore', 23: 'ecDaysbase', 24: 'age_B18+', 25: 'age_B43+', 26: 'age_less', 27: 'age_bottom', 28: 'age_top', 29: 'age_base', 30: 'age_B23+', 31: 'age_B28+', 32: 'age_B38+', 33: 'age_B33+', 34: 'age_B48+', 35: 'age_B53+', 36: 'age_B58+', 37: 'age_B63+', 38: 'age_B68+', 39: 'age_B73+', 40: 'age_B78+', 41: 'B', 42: 'Male', 43: 'Female', 44: 'A', 45: 'D', 46: 'C', 47: 'ecNum_7', 48: 'ecNum_23', 49: 'ecNum_34', 50: 'ecNum_22', 51: 'ecNum_87', 52: 'ecNum_26', 53: 'ecNum_25', 54: 'ecNum_1'}, 'v2idx'

### Adjust to dp Level

In [17]:
tensor_cols = [i for i in df_tensor.columns if 'idx' == i[-3:] or 'wgt' == i[-3:]]
final_cols = [R + 'ID' for R in RecName_Chain] + tensor_cols
df_tensor = df_tensor[final_cols]
df_tensor

,PID,P@PECInfo-InfoGrn_wgt,P@PECInfo-InfoGrn_keyidx,P@PECInfo-InfoGrn_tpcidx
0,P0,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,P1,"[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,P2,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,P3,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,P4,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
5,P5,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
6,P6,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
7,P7,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [18]:
from recfldgrn.graintools import get_highorder_input_idx
from functools import reduce


df_list = []
prefix_ids = [R + 'ID' for R in RecName_Chain][:-1]
focal_ids = [R + 'ID' for R in RecName_Chain][-1:]
for recfldgrn_sfx in tensor_cols:
    df_p = get_highorder_input_idx(df_tensor, recfldgrn_sfx, prefix_ids, focal_ids)
    df_list.append(df_p)
    
# Merge the dataframes in the list using reduce
df_tensor_fnl = reduce(lambda left, right: pd.merge(left, right), df_list)
df_tensor_fnl

,PID,P@PECInfo-InfoGrn_wgt,P@PECInfo-InfoGrn_keyidx,P@PECInfo-InfoGrn_tpcidx
0,P0,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,P1,"[0, 0, 0, 0, 0, 1, 0.6, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,P2,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,P3,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,P4,"[0, 0, 0, 0, 0, 1, 1.0, 0.0, 0.0, 0.0, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
5,P5,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
6,P6,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 0.2, 0.0, 0....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
7,P7,"[0, 0, 0, 0, 0, 1, 1.0, 1.0, 1.0, 1.0, 1.0, 1....","[1, 6, 26, 27, 28, 29, 24, 30, 31, 33, 32, 25,...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [19]:
prefix_recs = [i.replace('ID', '') for i in prefix_ids]
full_recfldgrn = '-'.join(prefix_recs) + '-' + recfldgrn if len(prefix_recs) > 0 else recfldgrn
full_recfldgrn

'P@PECInfo-InfoGrn'

### Save to Dataset

In [20]:
# from recfldgrn.datapoint import write_df_to_folders, RANGE_SIZE

# fldgrnTensor_folder = f'data/ProcData/FldGrnTensor/{full_recfldgrn}'
# if not os.path.exists(fldgrnTensor_folder): os.makedirs(fldgrnTensor_folder)

# print(recfldgrn)
# write_df_to_folders(recfldgrn, fldgrnTensor_folder, df_tensor_fnl, IDName)